# Optimized Solution:

The typical computation process is sequential i.e. task are assigned in queues and it is surved according to its arrival order LIFO. The normal core CPU can compute its fastly but the limitation of this approach is that it serves other process after completing the ones it sees first and CPU has limited numbers of threads. On the other hand GPU has handrads/thousands of thread that can compute parrallely. The following workflow will give more insight:

**CPU approach:**
<img src = 'https://raw.githubusercontent.com/hamzafar/GPU_Computation/master/images/sequential%20programming.PNG', width = 600>

**GPU approach:**
<img src = 'https://raw.githubusercontent.com/hamzafar/GPU_Computation/master/images/GPU%20procesing.PNG', width = 600>


# Compare performance 

To validate above we have compared the performance between sequential programming (CPU) and parallel programming (GPU) in terms of time.

Random data is generated and then simply multiplied using simple approach (for loop) and same data is multiplied on GPU in tensorflow multipy function.

The results shows that with simple arithmetic operations GPU outperformed.

### Random Data

In [1]:
import numpy as np
import time

import tensorflow as tf

In [2]:
# Number of rows to be multiplied
N = 50000

In [ ]:
# Random data genrated for multiplication with different seed
np.random.seed(10)
df = np.random.randn(N)

np.random.seed(100)
fms = np.random.randn(N)

np.random.seed(1000)
gff = np.random.randn(N)

np.random.seed(10000)
cof = np.random.randn(N)

pof = np.ndarray(N)
risk = np.ndarray(N)

### Sequential Programming

In [8]:
# determine the start time
start_time = time.time()

# multiply each of the variable
for i in range(0, len(cof)):
    pof[i] = df[i] * fms[i] * gff[i]

for i in range(0, len(pof)):
    risk[i] = pof[i] * cof[i]
# find out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08100461959838867 seconds ---


## Parallel Programing

In [9]:
# place data to tensors
t_df = tf.constant(df, name= 'df')
t_fms = tf.constant(fms, name= 'fms')
t_gff = tf.constant(gff, name= 'gff')
t_cof = tf.constant(cof, name= 'cof')

In [10]:
# multiply tensors
t_pof = tf.multiply(tf.multiply(t_df, t_fms), t_gff , name='pof')
t_risk = tf.multiply(t_pof, t_cof, name='cof')

In [11]:
# start tensorflow session
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [12]:
# determine the start time
start_time = time.time()

sess.run(t_pof)
sess.run(t_risk)
# find out the runtime
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.031001806259155273 seconds ---


In [13]:
sess.close()